<a href="https://colab.research.google.com/github/nandyc/rag_basic/blob/main/rag_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import userdata

In [20]:
# github_email = userdata.get('GITHUB_EMAIL')
# github_username = userdata.get('GITHUB_USERNAME')

In [28]:
!git config --global user.email {github_email}
!git config --global user.name {github_username}

In [29]:
!git clone https://github.com/nandyc/rag_basic.git

Cloning into 'rag_basic'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 19.49 KiB | 464.00 KiB/s, done.
Resolving deltas: 100% (4/4), done.


In [30]:
%cd rag_basic

/content/rag_basic/rag_basic


In [32]:
# !git status
# !echo "# Some dummy text" >> new.md
# !git add .
# !git commit -m "test"
# !git push origin main


On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	new.md

nothing added to commit but untracked files present (use "git add" to track)


[main 4917b04] test
 1 file changed, 1 insertion(+)
 create mode 100644 new.md
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
import os
import collections
from dotenv import load_dotenv
from openai import OpenAI
import chromadb
from chromadb.utils import embedding_functions

In [ ]:
%%capture
# !pip install python-dotenv
!pip install openai
!pip install chromadb
!pip install tiktoken

In [ ]:
# load_dotenv()
# openai_key =os.getenv("OPENAI_API_KEY")

False

In [ ]:
openai_key =userdata.get('OPENAI_APIKEY')

In [ ]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai_key, model_name="text-embedding-3-small"
)

In [ ]:
# Initialize the Chroma client with persistence
chroma_client = chromadb.PersistentClient(path="chroma_persistent_storage")
collection_name = "document_qa_collection"
collection = chroma_client.get_or_create_collection(
    name=collection_name, embedding_function=openai_ef
)

In [ ]:
client = OpenAI(api_key=openai_key)

In [ ]:
# resp = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {
#             "role": "user",
#             "content": "What is human life expectancy in the United States?",
#         },
#     ],
# )
# print(resp.choices[0].message.content)
# print (resp)

As of 2021, the average life expectancy in the United States is around 78.9 years. However, this can vary based on factors such as gender, race, and socio-economic status. It is important to note that life expectancy is an average and individual outcomes can vary.
ChatCompletion(id='chatcmpl-A0Ob84paAz1J8OtPVTTPcv1gj7EVi', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='As of 2021, the average life expectancy in the United States is around 78.9 years. However, this can vary based on factors such as gender, race, and socio-economic status. It is important to note that life expectancy is an average and individual outcomes can vary.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1724657918, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=58, prompt_tokens=27, total_tokens=85))


In [ ]:
# Function to load documents from a directory
def load_documents_from_directory(directory_path):
    print("==== Loading documents from directory ====")
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            with open(
                os.path.join(directory_path, filename), "r", encoding="utf-8"
            ) as file:
                documents.append({"id": filename, "text": file.read()})
    print(type(documents[0]))
    # list_of_filenames = collections.Counter([i['id'] for i in documents]).keys()
    list_of_filenames = [item['id'] for item in documents]
    print(list_of_filenames)
    return documents

In [ ]:
    # Function to split text into chunks
def split_text(text, chunk_size=1000, chunk_overlap=20):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - chunk_overlap
    print(chunks)
    return chunks

In [ ]:
    # Load documents from the directory
directory_path = "./news_articles"
documents = load_documents_from_directory(directory_path)

In [ ]:
print(f"Loaded {len(documents)} documents")

In [ ]:
# Split documents into chunks
chunked_documents = []
for doc in documents:
    chunks = split_text(doc["text"])
    print("==== Splitting docs into chunks ====")
    for i, chunk in enumerate(chunks):
        chunked_documents.append({"id": f"{doc['id']}_chunk{i+1}", "text": chunk})

print(f"Split documents into {len(chunked_documents)} chunks")
# print(chunked_documents)

In [ ]:
# Function to generate embeddings using OpenAI API
def get_openai_embedding(text):
    response = client.embeddings.create(input=text, model="text-embedding-3-small")
    embedding = response.data[0].embedding
    print("==== Generating embeddings... ====")
    return embedding

In [ ]:
# Generate embeddings for the document chunks
for doc in chunked_documents:
    print("==== Generating embeddings... ====")
    doc["embedding"] = get_openai_embedding(doc["text"])

# print(doc["embedding"])
print(chunked_documents)

In [ ]:
# Upsert documents with embeddings into Chroma
# for doc in chunked_documents:
#     print("==== Inserting chunks into db;;; ====")
#     collection.upsert(
#         ids=[doc["id"]], documents=[doc["text"]], embeddings=[doc["embedding"]]
#     )